In [2]:
import pandas as pd
import scipy.io as sio
from scipy.io import savemat, loadmat
from EEG_pareidolia_utils import get_pareidolia_bids, reformat_events
from biotuner_utils import *
from biotuner_offline import *
from biotuner import *
import pyACA
import mne
from mne.datasets.brainstorm import bst_raw
from mne.time_frequency import tfr_morlet, psd_multitaper, psd_welch
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import scipy.io as sio
import seaborn as sbn
import pytuning as pt

C:\Users\Antoine\anaconda3\lib\site-packages\sympy\__init__.py:672: SymPyDeprecationWarning: 

importing sympy.core.power with 'from sympy import *' has been
deprecated since SymPy 1.6. Use import sympy.core.power instead. See
https://github.com/sympy/sympy/issues/18245 for more info.

  self.Warn(


In [3]:
#path = '../../pareidolia_run4.fif'
path = 'C:/Users/Antoine/github/Data_EEG/pareidolia_run1.fif'
epochs = mne.read_epochs(path)
epochs = epochs.copy().apply_baseline((-1.5, -0.1))
#epochs2 = epochs2.crop(0.5, 7.5)
epochs = epochs.filter(0.1, 100)

Reading C:/Users/Antoine/github/Data_EEG/pareidolia_run1.fif ...


<ipython-input-3-f015be7621b3>:3: RuntimeWarning: This filename (C:/Users/Antoine/github/Data_EEG/pareidolia_run1.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs = mne.read_epochs(path)


    Found the data of interest:
        t =   -1500.00 ...    8000.00 ms
        0 CTF compensation matrices available
Not setting metadata
Not setting metadata
104 matching events found
No baseline correction applied
0 projection items activated
Applying baseline correction (mode: mean)
Setting up band-pass filter from 0.1 - 1e+02 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.10
- Lower transition bandwidth: 0.10 Hz (-6 dB cutoff frequency: 0.05 Hz)
- Upper passband edge: 100.00 Hz
- Upper transition bandwidth: 25.00 Hz (-6 dB cutoff frequency: 112.50 Hz)
- Filter length: 33001 samples (33.001 sec)



<ipython-input-3-f015be7621b3>:6: RuntimeWarning: filter_length (33001) is longer than the signal (9501), distortion is likely. Reduce filter length or filter a longer signal.
  epochs = epochs.filter(0.1, 100)


In [10]:
epochs.get_data()[:, 0, :].shape

(84, 9501)

In [130]:
#####COMPUTE PEAKS#####

from biotuner_offline import *
par = ['Image_on_par_high', 'Image_on_par_mid', 'Image_on_par_low']
nopar = ['Image_on_nopar_high', 'Image_on_nopar_mid', 'Image_on_nopar_low']
condition = par
ch = ['E10']
dim_reduc = 'avg' #Mode could either be 'avg' or 'mode'
precision = 0.25   #Default precision is 0.125Hz
FREQ_BANDS = [[1, 3], [3, 7], [7, 12], [12, 18],[18, 30], [30, 50]]
#peaks= compute_peaks(epochs.copy(), condition, ch, FREQ_BANDS, tmin = 0.5, tmax = 7.5, precision = precision, sf =1000)
peaks= compute_peaks_avg(epochs.copy(), condition, ch, FREQ_BANDS, dim_reduc = dim_reduc, tmin = 0.5, tmax = 7.5, precision = precision, sf =1000)
peaks = peaks.tolist()
peaks

Applying baseline correction (mode: mean)
(104, 114, 7001)
7000
4000
Effective window size : 4.000 (s)
5
Effective window size : 4.000 (s)
16
Effective window size : 4.000 (s)
11
Effective window size : 4.000 (s)
18
Effective window size : 4.000 (s)
48
Effective window size : 4.000 (s)
4


[2.2, 7.0, 9.8, 16.5, 30.0, 31.0]

In [131]:
peaks.insert(0, 1)
peaks_euler = [int(x*10) for x in peaks]
euler(peaks_euler[0], peaks_euler[1], peaks_euler[2], peaks_euler[3], peaks_euler[4], peaks_euler[5], peaks_euler[6])

65

[10, 22, 70, 98, 165, 300, 310]

In [ ]:
x, ratios = compute_peak_ratios(peaks)

In [60]:
peaks[1:]

array([ 3. ,  7.2, 16.8, 27.2, 32.2])

In [61]:
EEG_harmonics_mult(peaks[1:], 20, n_oct_up = 0)

array([[  3. ,   6. ,   9. ,  12. ,  15. ,  18. ,  21. ,  24. ,  27. ,
         30. ,  33. ,  36. ,  39. ,  42. ,  45. ,  48. ,  51. ,  54. ,
         57. ,  60. ,  63. ],
       [  7.2,  14.4,  21.6,  28.8,  36. ,  43.2,  50.4,  57.6,  64.8,
         72. ,  79.2,  86.4,  93.6, 100.8, 108. , 115.2, 122.4, 129.6,
        136.8, 144. , 151.2],
       [ 16.8,  33.6,  50.4,  67.2,  84. , 100.8, 117.6, 134.4, 151.2,
        168. , 184.8, 201.6, 218.4, 235.2, 252. , 268.8, 285.6, 302.4,
        319.2, 336. , 352.8],
       [ 27.2,  54.4,  81.6, 108.8, 136. , 163.2, 190.4, 217.6, 244.8,
        272. , 299.2, 326.4, 353.6, 380.8, 408. , 435.2, 462.4, 489.6,
        516.8, 544. , 571.2],
       [ 32.2,  64.4,  96.6, 128.8, 161. , 193.2, 225.4, 257.6, 289.8,
        322. , 354.2, 386.4, 418.6, 450.8, 483. , 515.2, 547.4, 579.6,
        611.8, 644. , 676.2]])

In [67]:
harmonic_fit(peaks[:], 30, 0.2, 'mult')

[6.55, 30.8, 61.6, 130.0, 156.0, 60.0, 120.0, 180.0, 240.0, 300.0]

In [45]:
peaks[1:]

array([ 3.2,  9.5, 13. , 25.8, 31.8])

In [77]:
#ratios of harmonic fits

z, charm_fit = compute_peak_ratios(harmonic_fit(peaks[1:], 30, 0.1, 'mult'))
charm_fit

list_remove = []
for r in range(len(charm_fit)-1):
    if charm_fit[r+1]-charm_fit[r] < 0.01:
        list_remove.append(charm_fit[r])        
charm_fit = [ele for ele in charm_fit if ele not in list_remove] 
    
charm_fit.insert(0, 1)
    
charm_fit

[1,
 1.2777777777777777,
 1.2916666666666667,
 1.3333333333333333,
 1.3478260869565217,
 1.5,
 1.7222222222222223,
 1.9166666666666667,
 1.9375,
 1.978494623655914,
 2.0]

In [62]:
peaks

array([ 2.2,  6.5, 10. , 12. , 21.5, 30.8])

In [66]:
from pytuning import *
intervals     = peaks[3:]
multiplicities = [1,2,1,3,2]
scale1        = create_euler_fokker_scale(intervals, multiplicities)
scale1

[1,
 19845/16384,
 315/256,
 1323/1024,
 21/16,
 45/32,
 3/2,
 6615/4096,
 441/256,
 945/512,
 15/8,
 63/32,
 2]

In [93]:
peaks

array([ 1. ,  3. ,  7.2, 16.8, 27.2, 32.2])

In [56]:
x = pt.find_best_modes(scale1, 4)
x

[{'scale': [1, 5/4, 3/2, 15/8, 2],
  'mask': (0, 6, 12, 18, 20),
  'steps': [6, 6, 6, 2],
  'original_scale': [1,
   135/128,
   9/8,
   4725/4096,
   75/64,
   315/256,
   5/4,
   21/16,
   675/512,
   2835/2048,
   45/32,
   189/128,
   3/2,
   1575/1024,
   105/64,
   14175/8192,
   225/128,
   945/512,
   15/8,
   63/32,
   2],
  'sum_p_q': 42,
  'sum_distinct_intervals': 8,
  'metric_3': 8.14285714285714,
  'sum_p_q_for_all_intervals': 107,
  'sum_q_for_all_intervals': 45}]

In [57]:
create_SCL(scale1, 'eeg_euler_fokker-01')

In [86]:
peaks_euler = [int(x*100) for x in peaks]
euler(peaks_euler[1], peaks_euler[2], peaks_euler[3], peaks_euler[4], peaks_euler[5])

201

In [78]:
##EULER##

from fractions import Fraction
from math import log
from functools import reduce
def gcd(*numbers):
    """
    Return the greatest common divisor of the given integers
    The result will have the same sign as the last number given (so that
    when the last number is divided by the result, the result comes out
    positive).
    """
    def gcd(a, b):
        while b:
            a, b = b, a % b
        return a

    return reduce(gcd, numbers)
def reduced_form(*numbers):
    """
    Return a tuple of numbers which is the reduced form of the input,
    which is a list of integers
    """
    return tuple(int(a // gcd(*numbers)) for a in numbers)
def lcm(*numbers):
    """
    Return the least common multiple of the given integers
    """
    def lcm(a, b):
        if a == b == 0:
            return 0
        return (a * b) // gcd(a, b)

    # LCM(a, b, c, d) = LCM(a, LCM(b, LCM(c, d)))
    return reduce(lcm, numbers)
def prime_factors(n):
    """
    Return a list of the prime factors of the integer n.
    Don't use this for big numbers; it's a dumb brute-force method.
    """
    factors = []
    lastresult = n

    while lastresult > 1:
        c = 2
        while lastresult % c > 0:
            c += 1
        factors.append(c)
        lastresult /= c

    return factors
def euler(*numbers):
    """
    Euler's "gradus suavitatis" (degree of sweetness) function
    Return the "degree of sweetness" of a musical interval or chord expressed
    as a ratio of frequencies a:b:c, according to Euler's formula
    Greater values indicate more dissonance
    """
    factors = prime_factors(lcm(*reduced_form(*numbers)))
    return 1 + sum(p - 1 for p in factors)